In [1]:
import os
import re
import pandas as pd
from bokeh.plotting import figure, ColumnDataSource
from bokeh.models import Select, CustomJS
from bokeh.layouts import column
from bokeh.io import output_file, show
from bokeh.palettes import Category10

In [2]:
input_folder    = '../output_files'
output_folder   = '../output_files/widget'

In [3]:
data_folder             = '../data'
geo_colombia_info_file  = 'geoinfo_municipios_colombia.csv'

In [4]:
df_mun_info = pd.read_csv(f'{data_folder}/{geo_colombia_info_file}')[['COD_DEPTO','NOM_DEPART']].drop_duplicates()

In [5]:
dataframes = {}
dataframes_fitted = {}

# Define the columns you expect in your DataFrame
expected_columns = ['date', 'week', 'Alpha', 'Gamma', 'Mu', 'Delta', 'Omicron', 'Otros'] #, 'OmicronBAX'

dept_list = df_mun_info.COD_DEPTO.sort_values()[:-1]
for dept_code in dept_list:

    file_path           = f'{input_folder}/processed_data/{dept_code}_dominance_data.csv'
    file_path_fitted    = f'{input_folder}/fitted_data/{dept_code}_dominance_data.csv'

    df_dom          = pd.read_csv(file_path)
    df_dom_fitted   = pd.read_csv(file_path_fitted)
    
    # Convert the 'date' column to datetime
    df_dom['date']          = pd.to_datetime(df_dom['date'])
    df_dom_fitted['date']   = pd.to_datetime(df_dom_fitted['date'])

    # Reindex the DataFrame to have the expected columns, filling with 0 if they don't exist
    df_dom          = df_dom.reindex(columns=expected_columns, fill_value=0)
    df_dom_fitted    = df_dom_fitted.reindex(columns=expected_columns, fill_value=0)

    # Store the DataFrame in the dictionary
    dept_name = df_mun_info[df_mun_info.COD_DEPTO == dept_code].NOM_DEPART.tolist()[0]
    dataframes[dept_name]           = df_dom
    dataframes_fitted[dept_name]    = df_dom_fitted


In [6]:
# Convert DataFrames to ColumnDataSource for both raw and fitted data
sources = {name: ColumnDataSource(df) for name, df in dataframes.items()}
sources_fitted = {name: ColumnDataSource(df) for name, df in dataframes_fitted.items()}

# Initial ColumnDataSource for both raw and fitted data
initial_source = ColumnDataSource(dataframes['SATA FE DE BOGOTÁ D.C.'])
initial_source_fitted = ColumnDataSource(dataframes_fitted['SATA FE DE BOGOTÁ D.C.'])


# Create the figure with datetime x-axis
p = figure(x_axis_type='datetime', title="Dominance Data Over Time", width=800, height=400)

# Assuming 'date' is the name of your datetime column
date_column = 'date'

# List of columns to plot
columns_to_plot = ['Alpha', 'Gamma', 'Mu', 'Delta', 'Omicron', 'Otros']

# Check if we have enough colors, otherwise cycle the palette
if len(columns_to_plot) > len(Category10[10]):
    colors = Category10[10] * (len(columns_to_plot) // len(Category10[10]) + 1)
else:
    colors = Category10[10][:len(columns_to_plot)]

for i, column_name in enumerate(columns_to_plot):
    color = colors[i]
    p.line(x=date_column, y=column_name, source=initial_source_fitted, line_width=2, color=color, alpha=0.8, legend_label=column_name, name="fitted")
    p.scatter(x=date_column, y=column_name, source=initial_source, size=8, color=color, legend_label=column_name, name="raw")

# Add legend customization here if needed
p.legend.location = "top_left"
p.legend.click_policy = "hide"

# Select widget with JavaScript callback to handle both raw and fitted data sources
select = Select(title="Choose dataset:", value='SATA FE DE BOGOTÁ D.C.', options=list(sources.keys()))
select.js_on_change('value', CustomJS(args=dict(sources=sources, sources_fitted=sources_fitted, plot=p), code="""
    const selected = cb_obj.value;
    const new_source = sources[selected];
    const new_source_fitted = sources_fitted[selected];
    
    // Update the plot based on the selected dataset
    plot.renderers.forEach(renderer => {
        if (renderer.data_source && renderer.name) {
            let source_to_use = renderer.name === "fitted" ? new_source_fitted : new_source;
            Object.assign(renderer.data_source.data, source_to_use.data);
            renderer.data_source.change.emit();
        }
    });
"""))

# Layout
layout = column(select, p)

# Output to HTML
output_file(f'{output_folder}/dominance_widget.html')
show(layout)
